In [1]:
import sys
import os
from itertools import islice
import logging

import pandas as pd
import numpy as np
from collections import OrderedDict

In [2]:
from Bio import SeqIO
from Bio.Seq import Seq 
from Bio.SeqRecord import SeqRecord

In [3]:
sys.path.append('/Users/kbw29/Dropbox/Research/Neurobiology_PhD/Huang/Projects/CellReadR/Code/alCellREADR')
from py_modules import *

In [4]:
tdT_Seq = "atggtgagcaagggcgaggaggtcatcaaagagttcatgcgcttcaaggtgcgcatggagggctccatgaacggccacgagttcgagatcgagggcgagggcgagggccgcccctacgagggcacccagaccgccaagctgaaggtgaccaagggcggccccctgcccttcgcctgggacatcctgtccccccagttcatgtacggctccaaggcgtacgtgaagcaccccgccgacatccccgattacaagaagctgtccttccccgagggcttcaagtgggagcgcgtgatgaacttcgaggacggcggtctggtgaccgtgacccaggactcctccctgcaggacggcacgctgatctacaaggtgaagatgcgcggcaccaacttcccccccgacggccccgtaatgcagaagaagaccatgggctgggaggcctccaccgagcgcctgtacccccgcgacggcgtgctgaagggcgagatccaccaggccctgaagctgaaggacggcggccactacctggtggagttcaagaccatctacatggccaagaagcccgtgcaactgcccggctactactacgtggacaccaagctggacatcacctcccacaacgaggactacaccatcgtggaacagtacgagcgctccgagggccgccaccacctgttcctggggcatggcaccggcagcaccggcagcggcagctccggcaccgcctcctccgaggacaacaacatggccgtcatcaaagagttcatgcgcttcaaggtgcgcatggagggctccatgaacggccacgagttcgagatcgagggcgagggcgagggccgcccctacgagggcacccagaccgccaagctgaaggtgaccaagggcggccccctgcccttcgcctgggacatcctgtccccccagttcatgtacggctccaaggcgtacgtgaagcaccccgccgacatccccgattacaagaagctgtccttccccgagggcttcaagtgggagcgcgtgatgaacttcgaggacggcggtctggtgaccgtgacccaggactcctccctgcaggacggcacgctgatctacaaggtgaagatgcgcggcaccaacttcccccccgacggccccgtaatgcagaagaagaccatgggctgggaggcctccaccgagcgcctgtacccccgcgacggcgtgctgaagggcgagatccaccaggccctgaagctgaaggacggcggccactacctggtggagttcaagaccatctacatggccaagaagcccgtgcaactgcccggctactactacgtggacaccaagctggacatcacctcccacaacgaggactacaccatcgtggaacagtacgagcgctccgagggccgccaccacctgttcctgtacggcatggacgagctgtacaagtaa"

In [5]:
# RIblast requires sequences in upper case 
targetSeq = Seq(tdT_Seq.upper())
templateSeq = targetSeq.reverse_complement()

In [6]:
# generator function 
def sliceSeq(seq, K):
    itr = iter(seq)
    res = tuple(islice(itr, K))
    if len(res) == K:
        yield res    
    for ele in itr:
        res = res[1:] + (ele,)
        yield res

In [7]:
def screen_SD(template, length):
    # Add a's as padding until divisible by 21
    while (len(template) % length) != 0: template = template + "a"
    # calling generator function 
    res = [Seq("".join(ele)) for ele in sliceSeq(template, length)] 
    # Filtered for stop and start codons ... and containing 
    # Removes ducplicates
    return list(OrderedDict.fromkeys([chunk for chunk in res if (len(chunk.translate(to_stop = True)) == (length/3)) & ("M" not in chunk.translate(to_stop = True)) & ("W" in chunk.translate())]))

In [8]:
# Removes duplicates ... mostly to deal with repeats 
sd_filtered = screen_SD(templateSeq, length = 21)
len(sd_filtered)

93

In [9]:
useful_RIblast = higher_order.output_RIblast(targetSeq, sd_filtered)

In [14]:
top_SD_IE = useful_RIblast.sort_values("InteractionEnergy_1", axis = 0).dropna(subset = ['InteractionEnergy_1'])
top_SD_IE

,Sequence,AccessibilityEnergy_1,HybridizationEnergy_1,InteractionEnergy_1,BasePair_1,AccessibilityEnergy_2,HybridizationEnergy_2,InteractionEnergy_2,BasePair_2
43,AGCCCATGGTCTTCTTCTGCA,7.70115,-32.64,-24.93890,(0-20:1122-1102),7.70379,-32.64,-24.93620,(0-20:396-376)
41,CCTCCCAGCCCATGGTCTTCT,7.77234,-32.55,-24.77770,(0-20:1128-1108),7.77449,-32.55,-24.77550,(0-20:402-382)
42,CCCAGCCCATGGTCTTCTTCT,6.81697,-31.45,-24.63300,(0-20:1125-1105),6.82011,-31.45,-24.62990,(0-20:399-379)
27,AGCTTCAGGGCCTGGTGGATC,8.33519,-32.91,-24.57480,(0-20:459-439),5.77979,-17.17,-11.39020,(0-19:968-951)
28,TTCAGGGCCTGGTGGATCTCG,6.39248,-30.93,-24.53750,(0-20:456-436),4.52663,-13.76,-9.23337,(0-17:675-662)
...,...,...,...,...,...,...,...,...,...
64,AGGGCAGGGGGCCGCCCTTGG,17.12110,-31.01,-13.88890,(0-20:159-139),5.34520,-16.85,-11.50480,(3-16:15-2)
60,GGTGCTTCACGTACGCCTTGG,15.19070,-28.33,-13.13930,(0-20:219-199),3.51320,-11.66,-8.14680,(6-14:657-649)
61,GCTTCACGTACGCCTTGGAGC,22.85190,-35.83,-12.97810,(0-20:168-148),NaN,NaN,NaN,NaN
63,CGTACGCCTTGGAGCCGTACA,22.31890,-34.43,-12.11110,(0-20:162-142),NaN,NaN,NaN,NaN


In [13]:
top_SD_IE.head(20) 

,Sequence,AccessibilityEnergy_1,HybridizationEnergy_1,InteractionEnergy_1,BasePair_1,AccessibilityEnergy_2,HybridizationEnergy_2,InteractionEnergy_2,BasePair_2
43,AGCCCATGGTCTTCTTCTGCA,7.70115,-32.64,-24.9389,(0-20:1122-1102),7.703790,-32.64,-24.93620,(0-20:396-376)
41,CCTCCCAGCCCATGGTCTTCT,7.77234,-32.55,-24.7777,(0-20:1128-1108),7.774490,-32.55,-24.77550,(0-20:402-382)
42,CCCAGCCCATGGTCTTCTTCT,6.81697,-31.45,-24.6330,(0-20:1125-1105),6.820110,-31.45,-24.62990,(0-20:399-379)
27,AGCTTCAGGGCCTGGTGGATC,8.33519,-32.91,-24.5748,(0-20:459-439),5.779790,-17.17,-11.39020,(0-19:968-951)
28,TTCAGGGCCTGGTGGATCTCG,6.39248,-30.93,-24.5375,(0-20:456-436),4.526630,-13.76,-9.23337,(0-17:675-662)
45,TGGTCTTCTTCTGCATTACGG,7.33911,-31.80,-24.4609,(0-20:1119-1099),7.340680,-31.80,-24.45930,(0-20:393-373)
6,TGGCGGCCCTCGGAGCGCTCG,4.90088,-28.64,-23.7391,(0-20:1329-1309),4.963850,-28.64,-23.67620,(0-20:603-583)
22,GCTTCTTGGCCATGTAGATGG,9.27296,-32.80,-23.5270,(0-20:676-656),9.809590,-18.41,-8.60041,(5-19:1250-1236)
44,CCATGGTCTTCTTCTGCATTA,9.90885,-33.35,-23.4411,(0-20:670-650),5.658940,-13.79,-8.13106,(10-18:999-991)
7,GCTCGTACTGTTCCACGATGG,5.06720,-28.19,-23.1228,(0-20:1326-1306),5.123940,-28.19,-23.06610,(0-20:600-580)


In [11]:
# Extract starting location of binding of sesRNA
[int(row.split(':', 1)[1].split('-', 1)[0]) for row in top_SD_IE['BasePair_1']]

[1122,
 1128,
 1125,
 459,
 456,
 1119,
 1329,
 676,
 670,
 1326,
 570,
 1323,
 1393,
 679,
 673,
 1320,
 1116,
 1405,
 861,
 862,
 66,
 664,
 337,
 423,
 429,
 517,
 441,
 138,
 502,
 384,
 876,
 139,
 444,
 432,
 435,
 630,
 1066,
 648,
 78,
 558,
 75,
 450,
 1213,
 685,
 567,
 867,
 490,
 91,
 438,
 426,
 499,
 387,
 81,
 505,
 639,
 453,
 343,
 447,
 334,
 228,
 873,
 142,
 87,
 645,
 682,
 564,
 642,
 346,
 879,
 69,
 84,
 508,
 222,
 493,
 633,
 871,
 72,
 870,
 561,
 881,
 328,
 636,
 331,
 677,
 94,
 555,
 496,
 225,
 159,
 219,
 168,
 162,
 165]

In [ ]:
sd_filtered["TGG" in sd_filtered]

In [ ]:
len([x for x in sd_filtered if "TGGA" in x])

In [ ]:
[x for x in sd_filtered if "TTGGA" in x]

In [ ]:
len([x for x in sd_filtered if "TTGG" in x])

In [15]:
def screen_RS(template, length):
    # Add a's as padding until divisible by 21
    while (len(template) % length) != 0: template = template + "a"
    # calling generator function 
    res = [Seq("".join(ele)) for ele in sliceSeq(template, length)] 
    # Filtered for stop and start codons ... and containing 
    # Removes ducplicates
    return list(OrderedDict.fromkeys([chunk for chunk in res if (len(chunk.translate(to_stop = True)) == (length/3)) & ("M" not in chunk.translate(to_stop = True)) ]))

In [16]:
# Removes duplicates ... mostly to deal with repeats 
rs_filtered = screen_RS(templateSeq, length = 15)
len(rs_filtered)

645

In [17]:
rs_useful_RIblast = higher_order.output_RIblast(targetSeq, rs_filtered)

In [21]:
top_RS_IE = rs_useful_RIblast.sort_values("InteractionEnergy_1", axis = 0).dropna(subset = ['InteractionEnergy_1'])
top_RS_IE

,Sequence,AccessibilityEnergy_1,HybridizationEnergy_1,InteractionEnergy_1,BasePair_1,AccessibilityEnergy_2,HybridizationEnergy_2,InteractionEnergy_2,BasePair_2
324,TCACGGTCACCAGAC,2.22648,-23.50,-21.27350,(0-14:968-954),2.22665,-23.50,-21.27330,(0-14:242-228)
565,CCATGTTGTTGTCCT,2.15973,-22.59,-20.43030,(0-14:21-7),6.92619,-17.91,-10.98380,(2-14:832-820)
325,CACGGTCACCAGACC,1.90009,-22.26,-20.35990,(0-14:966-952),1.90061,-22.26,-20.35940,(0-14:240-226)
164,GGCCTGGTGGATCTC,6.07997,-26.32,-20.24000,(0-14:1130-1116),6.08170,-26.32,-20.23830,(0-14:404-390)
567,TGTTGTTGTCCTCGG,1.95891,-21.95,-19.99110,(0-14:19-5),7.19359,-20.38,-13.18640,(0-14:832-818)
...,...,...,...,...,...,...,...,...,...
535,CTCGTGGCCGTTCAT,3.67639,-11.95,-8.27361,(0-14:1329-1312),3.74201,-11.95,-8.20799,(0-14:603-586)
531,CGAACTCGTGGCCGT,7.06348,-15.18,-8.11652,(3-14:695-684),NaN,NaN,NaN,NaN
53,TCGTACTGTTCCACG,6.37736,-14.48,-8.10264,(0-14:684-672),NaN,NaN,NaN,NaN
11,CGTCCATGCCGTACA,5.85543,-13.92,-8.06457,(0-13:1208-1189),5.85340,-13.92,-8.06660,(0-13:482-463)


In [22]:
top_RS_IE.head(20)

,Sequence,AccessibilityEnergy_1,HybridizationEnergy_1,InteractionEnergy_1,BasePair_1,AccessibilityEnergy_2,HybridizationEnergy_2,InteractionEnergy_2,BasePair_2
324,TCACGGTCACCAGAC,2.22648,-23.50,-21.2735,(0-14:968-954),2.22665,-23.50,-21.27330,(0-14:242-228)
565,CCATGTTGTTGTCCT,2.15973,-22.59,-20.4303,(0-14:21-7),6.92619,-17.91,-10.98380,(2-14:832-820)
325,CACGGTCACCAGACC,1.90009,-22.26,-20.3599,(0-14:966-952),1.90061,-22.26,-20.35940,(0-14:240-226)
164,GGCCTGGTGGATCTC,6.07997,-26.32,-20.2400,(0-14:1130-1116),6.08170,-26.32,-20.23830,(0-14:404-390)
567,TGTTGTTGTCCTCGG,1.95891,-21.95,-19.9911,(0-14:19-5),7.19359,-20.38,-13.18640,(0-14:832-818)
566,CATGTTGTTGTCCTC,2.25284,-22.18,-19.9272,(0-14:20-6),7.09039,-19.37,-12.27960,(1-14:832-819)
570,TGTTGTCCTCGGAGG,2.15169,-21.94,-19.7883,(0-14:16-2),6.42262,-16.46,-10.03740,(0-11:829-818)
563,CGGCCATGTTGTTGT,2.12295,-21.91,-19.7870,(0-14:23-9),6.04950,-15.49,-9.44050,(4-14:832-822)
568,GTTGTTGTCCTCGGA,2.12537,-21.70,-19.5746,(0-14:18-4),7.20047,-19.99,-12.78950,(0-13:831-818)
571,GTTGTCCTCGGAGGA,2.23042,-21.72,-19.4896,(0-14:15-1),2.16862,-10.72,-8.55138,(0-10:822-812)


In [19]:
len(top_RS_IE.iloc[0,0])

15

In [ ]:
# Extract starting location of binding of sesRNA
[int(row.split(':', 1)[1].split('-', 1)[0]) for row in top_RS_IE['BasePair_1']]